In [ ]:
import time
import pyepic

In [ ]:
client = pyepic.EPICClient("TOKEN HERE")

In [ ]:
local_data = "~/my_data/"
remote_folder = "epic://my_data/"

In [ ]:
# Sync the local data to remote data, this will copy any new or updated files to EPIC
# data_callback method used for monitoring the sync, uploaded will be true if the file has been copied
# false if not.

def data_callback(from_name, to_name, uploaded):
    if uploaded:
        print(f"Copied {from_name} to {to_name}")
    else:
        print(f"Did not copy {from_name} to {to_name}")

client.data.sync(local_data, remote_folder, dryrun=False, callback=data_callback)

In [ ]:
# Optionally, check we are using the right version of zcfd
apps = client.catalog.list_applications(product_name="zcfd")
for app in apps:
    for version in app.versions:
        print(f"ID={version.id} {app.product.name}({version.version}). Available on queues {version.queue_ids}")


In [ ]:
# Set the ID of the zcfd version we will use
zcfd_application_id = 36

In [ ]:
# Optionally, check the details of the queue we want to use
clusters = client.catalog.list_clusters(application_id=zcfd_application_id)

for cluster in clusters:
    print(f"ID = {cluster.id}, Name = {cluster.name}")


In [ ]:
# Set the ID of the cluster we will use
cluster_id = 12

In [ ]:
# Create and submit a series of 10 zcfd jobs, 
# each run in the same folder on the same mesh but with different control files
# 24 partitions per job
from pyepic.applications.zcfd import ZCFDJob

job_array = []

for job_id in range(0, 3):
    # Create the jobs
    job_obj =  ZCFDJob(zcfd_application_id,
                           f"job_name_{job_id}",
                           "epic://my_data/",
                           f"control_file_{job_id}.py",
                           "mesh.h5",
                           cycles=1000,
                           restart=False,
                           partitions=24)
    
    # Submit the jobs to the selected cluster
    job_spec = job_obj.get_job_create_spec(cluster_id)
    
    # Submit the job and store the returned job object for future use
    job_array.append(client.job.submit(job_spec))



In [ ]:
# Simple monitor to check the jobs every 5 mins
finished = 0 
while finished <= 3:
    for job in job_array:
        job_details = client.job.get_details(job[0].id)
        print(f"Job ID: {job_details.id}, Status: {job_details.status}, Finished: {job_details.finished}")
        if job_details.finished:
            finished += 1
    time.sleep(60 * 5)
    


In [ ]:
# Check the contents of a log file
import io

# Create a new BytesIO object
log_file_0 = io.BytesIO()

# Download contents of epic file into log_file_0
client.data.download_file("epic://my_data/control_file_0.log", log_file_0)

# Do something with the data in memory
log_file_0.seek(0)
print(log_file_0.read())

